This notebook reads the generated base_graph, base_network, and od_base_graph; loops for each hazard map in the folder and runs the od analysis. This happens N times (N= # of flood maps)

In [22]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import pickle
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (
    HazardSection,
    NetworkConfigData,
    NetworkSection,
    OriginsDestinationsSection
)
from ra2ce.configuration.config_wrapper import ConfigWrapper
from ra2ce.analysis.analysis_config_wrapper import AnalysisConfigWrapper
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionBase, AnalysisSectionLosses, ProjectSection
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import AnalysisDamagesEnum
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.analysis.losses.multi_link_origin_closest_destination import MultiLinkOriginClosestDestination
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_wrapper import NetworkConfigWrapper
from ra2ce.network.graph_files.graph_files_collection import GraphFilesCollection
from ra2ce.network.graph_files.graph_file import GraphFile
from ra2ce.network.graph_files.network_file import NetworkFile


In [23]:
# Functions

def get_all_files(directory: str) -> list[Path]:
    """
    Get all file names in the specified directory.

    Args:
        directory (str): The path to the directory.

    Returns:
        List[str]: A list of file names in the directory.
    """
    p = Path(directory)
    return [file for file in p.iterdir() if file.is_file()]

def read_pickle(file_path: str):
    """
    Read a pickle file.

    Args:
        file_path (str): The path to the pickle file.

    Returns:
        The object stored in the pickle file.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

def read_gpkg_to_gdf(file_path: str, layer: str = None) -> gpd.GeoDataFrame:
    """
    Read a GeoPackage file into a GeoDataFrame.

    Args:
        file_path (str): The path to the GeoPackage file.
        layer (str, optional): The specific layer to read from the GeoPackage. If None, reads the default layer.

    Returns:
        gpd.GeoDataFrame: The GeoDataFrame created from the GeoPackage file.
    """
    # Read the geopackage file into a GeoDataFrame
    gdf = gpd.read_file(file_path, layer=layer)
    
    return gdf

In [24]:
_root_dir = Path(r'.\exposure_and_od_analysis')
_static_path = _root_dir.joinpath("static")
_output_path = _root_dir.joinpath("output")
_base_graph_dir = _static_path.joinpath('output_graph')

hazard_files = get_all_files(_static_path / "hazard")
hazard_crs = "EPSG:32736" # for the hackathon case => "EPSG:4326" 

In [25]:
for hazard_file in hazard_files:
    # Make the NetworkConfigWrapper
    _hazard_section = HazardSection(
        hazard_map=[hazard_file],
        hazard_id=None,
        hazard_field_name="waterdepth",
        aggregate_wl=AggregateWlEnum.MAX,
        hazard_crs=hazard_crs,
        overlay_segmented_network = False
    )

    _origin_destination_section = OriginsDestinationsSection(
        origins=_static_path.joinpath("network", "origins.shp"),
        destinations=_static_path.joinpath("network", "destinations.shp"),
        origins_names="A",
        destinations_names="B",
        id_name_origin_destination="OBJECTID",
        origin_count="POPULATION",
        origin_out_fraction=1,
        category="category",
    )
    
    _network_config_data = NetworkConfigData(
        root_path=_root_dir,
        static_path=_static_path,
        output_path=_output_path,
        hazard=_hazard_section,
        origins_destinations=_origin_destination_section,
    )    
    _network_config_data.network.save_gpkg = True

    # Make the AnalysisConfigWrapper
    _analysis_section = AnalysisSectionLosses(
        name='origin_closest_destination',
        analysis=AnalysisLossesEnum.MULTI_LINK_ORIGIN_CLOSEST_DESTINATION,
        aggregate_wl= AggregateWlEnum.MAX,
        threshold=0.5,
        weighing=WeighingEnum.LENGTH,
        calculate_route_without_disruption=True,
        save_gpkg=True,
        save_csv=True,
    )

    _analysis_config_data = AnalysisConfigData(
        project=ProjectSection(name=hazard_file.stem),
        root_path=_root_dir,
        static_path=_static_path,
        output_path=_output_path,
        analyses=[_analysis_section],
        
    )

    # Add the generated graphs
    _graph_files = GraphFilesCollection(
        base_graph=GraphFile(
            name="base_graph",
            folder=_base_graph_dir,
            graph=read_pickle(_base_graph_dir / "base_graph.p"),
            ),
        base_network=NetworkFile(
            name="base_network",
            folder=_base_graph_dir,
            graph=read_gpkg_to_gdf(_base_graph_dir / "base_network.gpkg"),
            ),
        origins_destinations_graph=GraphFile(
            name="origins_destinations_graph",
            folder=_base_graph_dir,
            graph=read_pickle(_base_graph_dir / "origins_destinations_graph.p"),
            ),
    )
    _handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)
    
    # Run analysis
    _handler.input_config.network_config.graph_files = _graph_files
    _handler.configure()
    _handler.run_analysis()

2024-07-18 06:40:53 PM - [hazard_overlay.py:381] - root - WARNING - Hazard crs EPSG:32736 and graph crs EPSG:4326 are inconsistent, we try to reproject the graph crs
2024-07-18 06:40:53 PM - [hazard_overlay.py:381] - root - WARNING - Hazard crs EPSG:32736 and graph crs EPSG:4326 are inconsistent, we try to reproject the graph crs
2024-07-18 06:40:53 PM - [hazard_overlay.py:381] - root - WARNING - Hazard crs EPSG:32736 and graph crs EPSG:4326 are inconsistent, we try to reproject the graph crs
2024-07-18 06:40:53 PM - [hazard_overlay.py:381] - root - WARNING - Hazard crs EPSG:32736 and graph crs EPSG:4326 are inconsistent, we try to reproject the graph crs
2024-07-18 06:40:53 PM - [hazard_overlay.py:381] - root - WARNING - Hazard crs EPSG:32736 and graph crs EPSG:4326 are inconsistent, we try to reproject the graph crs
2024-07-18 06:40:53 PM - [hazard_overlay.py:381] - root - WARNING - Hazard crs EPSG:32736 and graph crs EPSG:4326 are inconsistent, we try to reproject the graph crs
2024

IndexError: single positional indexer is out-of-bounds

In [ ]:
# Run analysis
_handler = Ra2ceHandler.from_config(None, _analysis_config_data)
_handler.configure()
_handler.run_analysis()

## Origin closest destination analyses

## Origin closest destination without hazard

Firstly, define the folder in which the data exist:

In [ ]:
# specify the path to the folder holding the RA2CE folder and input data
root_dir = Path ("data", "origin_destination_analysis_without_hazard")

In [ ]:
origins_inspection = root_dir/"static"/"network"/"origins.shp"
origins_gdf = gpd.read_file(origins_inspection, driver = "SHP")
origins_gdf.head()
origins_gdf.explore(column = "POPULATION", cmap = "viridis_r", tiles="CartoDB dark_matter")

In [ ]:
destinations_inspection = root_dir/"static"/"network"/"destinations.shp"
destinations_gdf = gpd.read_file(destinations_inspection, driver = "SHP")
destinations_gdf.explore(column = "category", cmap = ['Blue', 'Green'], tiles="CartoDB dark_matter")

In [ ]:
origins_gdf.head()

In [ ]:
destinations_gdf.head()

### Specifying the .ini files

Now, to tell RA2CE how to use the origins and destinations, the user needs to specify the file names in the network.ini.

*Notice how you can keep the [network] parameters the same as in the RA2CE Basics example. RA2CE will automatically use the graph you have already created. Or, if you want to create a new graph, it can run both processes consecutively.*

**Network.ini content**
<br>
<br>
> [project]<br>
name = beira<br>
<br>
[network]<br>
**directed = False<br>
source = OSM download<br>
primary_file = None<br>
diversion_file = None<br>
file_id = rfid_c<br>
polygon = region_polygon.geojson<br>
network_type = drive<br>
road_types = motorway,motorway_link,primary,primary_link,secondary,secondary_link,tertiary,tertiary_link,residential<br>
save_gpkg = True**<br>
<br>
[origins_destinations]<br>
**origins = origins.shp<br>
destinations = destinations.shp<br>
origins_names = A<br>
destinations_names = B<br>
id_name_origin_destination = OBJECTID<br>
origin_count = POPULATION<br>
origin_out_fraction = 1<br>
category = category**<br>
<br>
*[hazard]<br>
hazard_map = None<br>
hazard_id = None<br>
hazard_field_name = None<br>
aggregate_wl = None<br>
hazard_crs = None<br>
<br>
[cleanup]<br>
snapping_threshold = None<br>
segmentation_length = None<br>
merge_lines = True<br>
merge_on_id = False<br>
cut_at_intersections = False<br>*

We now need to update our analysis initialisation files using the preferred OD-analysis (there are multiple). For now we will consider the **optimal_route_origin_closest_destination** analysis. This analysis finds from each origin location, the closest destination per category. Another analysis that RA2CE offers is an OD analysis which tries to find routes from all origins to all destinations, but we won't run that one in this example. All types of losses analyses can be found [here](https://deltares.github.io/ra2ce/analysis_module/analysis_module.html#indirect-losses-network-criticality).

Navigate to the folder on your local machine with which you want to perform the analysis, and change the analysis.ini accordingly. Weighing defines the criterium based on which the optimal routes will be defined. It accepts *time* as well.

**Analysis.ini content**
<br>
<br>
> [project] <br>
name = beira <br>
<br>
[analysis1]<br>
name = multilink origin closest destination without hazard<br>
analysis = multi_link_origin_closest_destination<br>
aggregate_wl = None<br>
threshold = None<br>
weighing = distance<br>
calculate_route_without_disruption = True<br>
save_gpkg = True<br>
save_csv = True<br>
<br>

Set the paths to the initialization files and check if the files exist.

In [ ]:

_network_ini_name = "network.ini" #set the name for the network.ini
_analysis_ini_name = "analysis.ini" #set the name for the analysis.ini

network_ini = root_dir / _network_ini_name
assert network_ini.is_file()

analysis_ini = root_dir / _analysis_ini_name
assert analysis_ini.is_file()

Run RA2CE. Notice the information RA2CE gives you. Education locations are referred to as D1 (destination 1) and hospital locations as D2 (destination 2).

In [ ]:
handler = Ra2ceHandler(network=network_ini, analysis=analysis_ini)
handler.configure()
handler.run_analysis()

## Origin closest destination with hazard

Firstly, define the folder in which the data exist:

In [ ]:
# specify the path to the folder holding the RA2CE folder and input data
root_dir = Path (".\\data\\origin_destination_analysis_with_hazard")

Secondly, we need a flood map! We have an example flood map in our example folder. Let's inspect it.

*Note: the flood map needs to be in **.tif** format*

In [ ]:
hazard_folder = root_dir / "static" / "hazard" # find the hazard folder where you locate your floo dmap
hazard_map = hazard_folder / "max_flood_depth.tif" # set the location of the hazard map

# Open the TIF file using rasterio
with rasterio.open(hazard_map) as src:
    # Read the TIF file as a numpy array
    tif_array = src.read(1)  # Change the band index (1) if necessary

plt.figure(figsize=(10, 10))
plt.imshow(tif_array, cmap='Blues')  # Change the colormap if desired
plt.colorbar(label='Pixel Values')
plt.title('Flood map')
plt.show()


To use the flood map with RA2CE, we need to fill in the **[hazard]** section in the network.ini. 

Specify the flood map name in the **hazard_map** parameter in network.ini. RA2CE expects the flood map to be located in the *hazard* folder. The **aggregate_wl** parameter in analysis.ini can be set to either 'max', 'min' or 'mean' to take the maximum, minimum or mean water depth per road segment when the exposure of the roads to a certain hazard (map) is determined.<br>

Set the right CRS for the flood map in the **hazard_crs** parameter. This CRS can be different from the origins, destinations and road network. RA2CE will reproject the network to the CRS of the flood map and will reproject the road back to the original CRS when the CRS differs.

You can run a RA2CE analysis with only the hazard overlay on the roads. The results can be found in the output_graph folder. The data with *'\*_hazard'*, contains the result of the overlay with the hazard. Here, we will load the data from the example graph folder. 

*If you want to practice with this, create a new project folder where you specify the hazard map and perform the hazard analysis with RA2CE*

Notice the column **EV1_ma**. This refers to the hazard. This column holds the water depth of the road segment. 'EV1' stands for 'Event 1' (you can run multiple flood maps, the column results will be called EV1, EV2, EV3, etc.). '_ma' refers to maximum flood depth, which is the parameter specified in the network.ini. We always use the maximum water depth for the OD analysis because a vehicle can only use a road segment when it can drive through the largest water depth on that road segment.

### Specifying the .ini files

Now, to tell RA2CE how to use the origins and destinations, the user needs to specify the file names in the network.ini.

*Notice how you can keep the [network] parameters the same as in the RA2CE Basics example. RA2CE will automatically use the graph you have already created. Or, if you want to create a new graph, it can run both processes consecutively.*

**Network.ini content**
<br>
<br>
> [project]<br>
name = beira<br>
<br>
[network]<br>
**directed = False<br>
source = OSM download<br>
primary_file = None<br>
diversion_file = None<br>
file_id = rfid_c<br>
polygon = region_polygon.geojson<br>
network_type = drive<br>
road_types = motorway,motorway_link,primary,primary_link,secondary,secondary_link,tertiary,tertiary_link,residential<br>
save_gpkg = True**<br>
<br>
[origins_destinations]<br>
**origins = origins.shp<br>
destinations = destinations.shp<br>
origins_names = A<br>
destinations_names = B<br>
id_name_origin_destination = OBJECTID<br>
origin_count = POPULATION<br>
origin_out_fraction = 1<br>
category = category**<br>
<br>
[hazard]<br>
**hazard_map = max_flood_depth.tif<br>
hazard_id = None<br>
hazard_field_name = waterdepth<br>
aggregate_wl = max<br>
hazard_crs = EPSG:32736**<br>
<br>
*[cleanup]<br>
snapping_threshold = None<br>
segmentation_length = None<br>
merge_lines = True<br>
merge_on_id = False<br>
cut_at_intersections = False<br>*

We now need to update our analysis initialisation files using the preferred OD-analysis (there are multiple). For now we will consider the **optimal_route_origin_closest_destination** analysis. This analysis finds from each origin location, the closest accessible non-flooded destination per category. Another analysis that RA2CE offers is an OD analysis which tries to find routes from all origins to all destinations, but we won't run that one in this example. All types of losses analyses can be found [here](https://deltares.github.io/ra2ce/analysis_module/analysis_module.html#indirect-losses-network-criticality).

Navigate to the folder on your local machine with which you want to perform the analysis, and change the analysis.ini accordingly.

With the **aggregate_wl** parameter, the user can choose which type of aggregation of the water level on the road segment (max, mean, min) the analysis should consider. The **threshold** is the hazard intensity which determines road disruption, in this case it is set to 0.5, which relates to 0.5 m of water depth on the road segment. Weighing defines the criterium based on which the optimal routes will be defined. It accepts *time* as well. The analysis considers **distance** as measure for finding the optimal routes. With the parameter **calculate_route_without_disruption** set to True, RA2CE will calculate the routes with hazard disruption *and without*. Finally, with the parameters **save_gpkg** and **save_csv** the user can choose to save or not save resulting output shapefiles/gpkg (we are in a transition from shp to gpkg)/csvs.

**Analysis.ini content**
<br>
<br>
> [project] <br>
name = beira <br>
<br>
[analysis1]<br>
name = multi_link origin closest destination<br>
analysis = multi_link_origin_closest_destination<br>
aggregate_wl = max<br>
threshold = 0.5<br>
weighing = distance<br>
calculate_route_without_disruption = True<br>
save_gpkg = True<br>
save_csv = True<br>
<br>

Set the paths to the initialization files and check if the files exist.

In [ ]:

_network_ini_name = "network.ini" #set the name for the network.ini
_analysis_ini_name = "analysis.ini" #set the name for the analysis.ini

network_ini = root_dir / _network_ini_name
assert network_ini.is_file()

analysis_ini = root_dir / _analysis_ini_name
assert analysis_ini.is_file()

Run RA2CE. Notice the information RA2CE gives you. Education locations are referred to as D1 (destination 1) and hospital locations as D2 (destination 2).

In [ ]:
handler = Ra2ceHandler(network=network_ini, analysis=analysis_ini)
handler.configure()
handler.run_analysis()

## Inspecting results
Let's do some output exploration!

In [ ]:
analysis_output_path = root_dir / "output" / "multi_link_origin_closest_destination"
gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_destinations.gpkg')
gdf.head() #show the origins

Notice the different columns. Especially the columns 'EV1_ma_PD1' and 'EV1_ma_PD2' are of interest. They refer to 'EV1' (event 1), maximum water depth (ma) and Destination 1 (education) or 2 (hospital).

Below, we visualise which origins have access to their closest destination, given the disruption of the road network because of the flood. 

*Note: below we visualize the access to education (D1).*

In [ ]:
gdf_education = gdf[gdf['category']=='education']
gdf_education['access_PD1'] = gdf_education.apply(lambda row: '1' if row['EV1_ma_PD1'] > 0 else '0', axis=1)
gdf_education.explore(column='access_PD1', cmap=['red', 'green'], tiles="CartoDB dark_matter")

We can also check the access to the hospitals:

In [ ]:
gdf_hospital = gdf[gdf['category']=='hospital']
gdf_hospital['access_PD2'] = gdf_hospital.apply(lambda row: '1' if row['EV1_ma_PD2'] > 0 else '0', axis=1)
gdf_hospital.explore(column='access_PD2', cmap=['red', 'green'], tiles="CartoDB dark_matter")

What RA2CE also provides are optimal routes to a destination. In this case, since we use the origin **closest** destination analysis, the routes will be centered around the destination. For every origin that still has access, RA2CE computes the optimal route to that closest destination. 

If we want to visualize this in a way that shows the routes clearly, it requires some filtering. You for example need to filter the destination of interest. You can do this here or in a GIS software. The optimal routes are shown with a colour scheme which represents the temporal difference between the optimal routes with and without the hazard. 

In [ ]:
origin_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_origins.gpkg') # read in the origins
destinations_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_destinations.gpkg') # read in the destination
optimal_routes_with_hazard_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_optimal_routes_with_hazard.gpkg') # read in the optimal routes given the hazard

In [ ]:
b_4_gdf = destinations_gdf[destinations_gdf['d_id']=='B_4'] # filter on destination B4
optimal_routes_b_4_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['destination'] == 'B_4'] # filter on destination B4
origins_with_optimal_route_b_4 = origin_gdf[origin_gdf['o_id'].isin(optimal_routes_b_4_with_hazard_gdf['origin'])] # both the origins and the destinations file hold destination information

optimal_routes_b_4_with_harard_map = optimal_routes_b_4_with_hazard_gdf.explore(column='difference', tiles="CartoDB dark_matter")
b_4_map = b_4_gdf.explore(m=optimal_routes_b_4_with_harard_map, color='blue', marker_kwds={'radius':10}, tiles="CartoDB dark_matter")
origins_with_optimal_route_b_4.explore(m=b_4_map, color='green', marker_kwds={'radius':5}, tiles="CartoDB dark_matter")

Now lets visualise the optimal routes for the same destination for the situation before the occurance of the hazard. What are the differences with the after disaster situation?

In [ ]:
optimal_routes_without_hazard_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_optimal_routes_without_hazard.gpkg') # read in the optimal routes given the hazard

optimal_routes_b_4_without_hazard_gdf = optimal_routes_without_hazard_gdf[optimal_routes_without_hazard_gdf['destination'] == 'B_4'] # filter on destination B4
origins_without_optimal_route_b_4 = origin_gdf[origin_gdf['o_id'].isin(optimal_routes_b_4_without_hazard_gdf['origin'])] # both the origins and the destinations file hold destination information

optimal_routes_b_4_without_harard_map = optimal_routes_b_4_without_hazard_gdf.explore(color='white', tiles="CartoDB dark_matter")
b_4_map = b_4_gdf.explore(m=optimal_routes_b_4_without_harard_map, color='blue', marker_kwds={'radius':10}, tiles="CartoDB dark_matter")
origins_without_optimal_route_b_4.explore(m=b_4_map, color='green', marker_kwds={'radius':5}, tiles="CartoDB dark_matter")

Notice how some origins have access but are not directly on a road. This is becasuse RA2CE makes an assumption where the origins will 'enter' the road, by projecting the origin on the nearest road vertice. 

It is also interesting to see how many people are cut-off from their nearest education location or hospital. We first show the the number of people that does not have access to any education location.

In [ ]:
origins_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_origins.gpkg')
no_access_gdf = origins_gdf[origins_gdf['EV1_ma_AD1']=='no access']
no_access_gdf.explore(column='POPULATION', cmap='cool', marker_kwds={'radius':5}, tiles="CartoDB dark_matter")

And now the same for the hospitals:

In [ ]:
origins_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_origins.gpkg')
no_access_gdf = origins_gdf[origins_gdf['EV1_ma_AD2']=='no access']

no_access_gdf.explore(column='POPULATION', cmap='cool', marker_kwds={'radius':5}, tiles="CartoDB dark_matter")

These two maps almost look the same, are they? Are there just as many people isolated from any education location as from any hospital location? Let's check it in the results.

In [ ]:
origins_gdf = gpd.read_file(analysis_output_path / 'multi_link_origin_closest_destination_origins.gpkg')
any_d1_d2_access = origins_gdf[(origins_gdf['EV1_ma_AD1']=='access') | (origins_gdf['EV1_ma_AD2']=='access')]
any_d1_d2_no_access = origins_gdf[(origins_gdf['EV1_ma_AD1']=='no access') | (origins_gdf['EV1_ma_AD2']=='no access')]
d1_d2_access = origins_gdf[(origins_gdf['EV1_ma_AD1']=='access') & (origins_gdf['EV1_ma_AD2']=='access')]
d1_d2_no_access = origins_gdf[(origins_gdf['EV1_ma_AD1']=='no access') & (origins_gdf['EV1_ma_AD2']=='no access')]

print(f"How many population points have access to any educational or healthcare location? {any_d1_d2_access.shape[0]}")
print(f"How many population points have no access to either any educational or healthcare location? {any_d1_d2_no_access.shape[0]}")
print(f"How many population points have access to any educational and healthcare location? {d1_d2_access.shape[0]}")
print(f"How many population points have no access to any educational and healthcare location? {d1_d2_no_access.shape[0]}")

Indeed, as we already saw in the maps all locations have either access to an education location ánd to a hospital or no access to either.